# Ollama API 參數說明

## payload 字典參數詳解

### 基本參數
- **`model`**: 指定要使用的 AI 模型版本
  - 範例: `"llama3.2:latest"` - 使用 Llama 3.2 最新版本
  - 其他可用模型: `"gemma2:latest"`, `"qwen2:latest"` 等

- **`prompt`**: 輸入給 AI 的問題或指令
  - 這是您想要 AI 回答或執行的內容

- **`stream`**: 控制回應模式
  - `False`: 非串流模式，一次返回完整回應
  - `True`: 串流模式，即時返回部分回應

### 生成控制參數
- **`max_tokens`**: 限制 AI 回應的最大 token 數量
  - 約 1 token = 0.75 個英文單詞
  - 設定 `100` 表示回應約 75 個英文單詞
  - 控制回應長度，避免過長回應

- **`format`**: 指定回應格式
  - `"json"`: 要求 AI 以 JSON 格式回應
  - `"text"`: 純文字格式（預設）

### options 子參數（生成品質控制）

#### Temperature（溫度參數）
- **範圍**: 0.0 - 1.0
- **作用**: 控制回應的創造性和隨機性
- **範例值**: `0.7`
  - `0.0`: 完全確定性，總是給出相同回應
  - `0.3`: 低創造性，回應較為保守
  - `0.7`: 平衡創造性和一致性（推薦）
  - `1.0`: 高創造性，回應變化很大

#### Top-p（核採樣）
- **範圍**: 0.0 - 1.0
- **作用**: 控制詞彙選擇範圍
- **範例值**: `0.9`
  - `0.9`: 只考慮累積機率前 90% 的詞彙
  - 較高值 = 更多詞彙選擇，回應更多樣
  - 較低值 = 更聚焦的詞彙選擇，回應更一致

#### Top-k（頂部 K 採樣）
- **範圍**: 1 - 1000+
- **作用**: 限制每次只考慮機率最高的前 K 個詞彙
- **範例值**: `50`
  - 防止選擇機率過低的詞彙
  - 提高回應品質和一致性
  - 較大值 = 更多選擇，較小值 = 更聚焦

## 參數調優建議

### 創意寫作場景
```python
"options": {
    "temperature": 0.8,  # 高創造性
    "top_p": 0.95,       # 更多詞彙選擇
    "top_k": 100,        # 更多選擇
}
```

### 技術問答場景
```python
"options": {
    "temperature": 0.3,  # 低創造性
    "top_p": 0.8,        # 較聚焦
    "top_k": 30,         # 較少選擇
}
```

### 平衡場景（推薦）
```python
"options": {
    "temperature": 0.7,  # 平衡
    "top_p": 0.9,        # 平衡
    "top_k": 50,         # 平衡
}
```


In [5]:
import requests

def chat_with_ollama(prompt: str):
    url = "http://localhost:11434/api/generate"
    payload = {
        "model": "gemma3:1b",
        "prompt": prompt,
        "stream": False,
        "options": { #參考說明1
            "temperature": 0.7,
            "top_p": 0.9,
            "top_k": 50,
        },
        "max_tokens": 1000,
        "format": "json",
    }

    response = requests.post(url, json=payload)
    result = response.json()
    print("💬 AI 回應：")
    # Print the whole result for debugging
    print(result)
    # Try to print the 'response' key if it exists, otherwise print possible keys
    if "response" in result:
        print(result["response"])
    elif "message" in result:
        print(result["message"])
    elif "content" in result:
        print(result["content"])
    else:
        print("No expected key found in response. Available keys:", result.keys())

#範例輸入
chat_with_ollama("請用簡單的方式解釋什麼是Python的函式？")

💬 AI 回應：
{'model': 'gemma3:1b', 'created_at': '2025-09-13T08:00:38.0722077Z', 'response': '{\n"解釋：":"Python的函式就像小工具，你可以用來做一些特定的事情。你可以把它們想像成一個指令，告訴 Python 執行什麼。 簡單來說，函式會執行一個特定的操作，並會返回結果。"\n}', 'done': True, 'done_reason': 'stop', 'context': [105, 2364, 107, 239230, 237105, 74624, 48280, 185411, 26549, 237026, 32651, 236918, 238780, 237522, 237536, 106, 107, 105, 4368, 107, 236782, 107, 236775, 185411, 237184, 12375, 32651, 236918, 238780, 237522, 55326, 237369, 30398, 236900, 71407, 237105, 237967, 237893, 18303, 172612, 25606, 236924, 71407, 91033, 238050, 103146, 237283, 19966, 81582, 236900, 146132, 17856, 236743, 78204, 26549, 236924, 236743, 74624, 94757, 236900, 238780, 237522, 238003, 78204, 19966, 172612, 17553, 236900, 238953, 238003, 29153, 27654, 165220, 107, 236783], 'total_duration': 1670237400, 'load_duration': 124511400, 'prompt_eval_count': 21, 'prompt_eval_duration': 109436000, 'eval_count': 58, 'eval_duration': 1298664500}
{
"解釋：":"Python的函式就像小工具，你可以用來做一些特定的事情。你可以把它們想像